In [1]:
import polars as pl
import polars.selectors as cs
import altair as alt
from helper import pivot_txs, check_for_orphaned_txids, make_transitions, make_tx_stats_chart
pl.Config.set_tbl_rows(7)
pl.Config.set_fmt_str_lengths(64)  # txid's are 64 characters

polars.config.Config

### Read in dataframes

In [2]:
tx_inputs = pl.read_parquet('./dfs/transaction_inputs.parquet')
tx_outputs = pl.read_parquet('./dfs/transaction_outputs.parquet')
txs = pl.read_parquet('./dfs/transactions.parquet')
session_times = pl.read_parquet('./dfs/session_times.parquet')

### Pivot input and output dfs so 1 row = 1 `txid` in each df

In [3]:
# Make a wide (columns = txid + 4 kind's) input tx table
groupby_cols = ['txid', 'kind', 'ln_contract_id']
p_in = pivot_txs(tx_inputs, groupby_cols)

In [4]:
# Make a wide (columns = txid + 4 kind's) output tx table
groupby_cols = ['txid', 'kind', 'ln_contract_interaction_kind', 'ln_contract_id']
p_out = pivot_txs(tx_outputs, groupby_cols)

In [5]:
# Make a combine wide (columns = txid + 4 input kind's + 4 output kind's) input tx table
in_cols = ['ln', 'mint', 'stability_pool', 'wallet']
suffix = '_out' 
out_cols = [f'{col}{suffix}' for col in in_cols]
p_combined = p_in.join(p_out, on='txid', suffix=suffix).select(['txid'] + in_cols + out_cols)

### Investigate `kind` values

#### Observation: All input `txid`s have a single `kind` value.

In [6]:
cols = ['ln', 'mint', 'stability_pool', 'wallet']
print('Input txs with multiple kinds:')
display(p_in.filter(pl.sum_horizontal(pl.col(cols).is_not_null()) > 1))

Input txs with multiple kinds:


txid,ln,mint,stability_pool,wallet
str,i64,i64,i64,i64


#### Observation: Some output `txid`s have multiple `kind` values.

In [7]:
cols = ['ln', 'mint', 'stability_pool', 'wallet']
print('\nOutput txs with multiple kinds:')
display(p_out.filter(pl.sum_horizontal(pl.col(cols).is_not_null()) > 1))


Output txs with multiple kinds:


txid,ln,mint,stability_pool,wallet
str,i64,i64,i64,i64
"""00d487990553f7b732419e61b9cd28e73d1e7346a8642d17773188005e5b6f30""",41493425,449615,null,null
"""00e23fc71b51d705366029e78b3628be0af4ed246fd388e27717dee0661f5e31""",27813365,6667,null,null
"""0333a5b127ed2d8f278917abe9b6ef26f62ca9aacbb000407cfadefa276b7d80""",5000000,5312,null,null
"""03f4b2e925d681aa1e71bd24d95425bbb98ac2c6e18895e0b38de58606c9201f""",102500,28572,null,null
…,…,…,…,…
"""fbfd34210bc0bf82957999639d19a1775fe7250303c59b8f544c333a242068c7""",9035,181,null,null
"""fd6d608a8033d139bed7218876d01e2b8ba9b90551e9e096950e1831e8e7f029""",102500,28572,null,null
"""ff3e50deaf9498be67705d2b7aef0acac1bf97ea072c62fe3d966df514f28dfa""",102500,28572,null,null


#### Observation: When multiple `kind` values exist, `mint` is involved on both sides (input and output).

In [8]:
with pl.Config(tbl_rows=20):
    display(make_transitions(p_combined).sort(by='in_msat', descending=True))

in,out,n,in_msat,out_msat,diff
str,str,u32,i64,i64,i64
"""wallet""","""mint""",63,842375871000,842375871000,0
"""mint""","""wallet""",45,718674952000,718674952000,0
"""mint""","""mint""",2935,709206199392,709206199392,0
"""ln""","""mint""",66913,118318101255,118318101255,0
"""mint""","""ln""",66508,91649015375,91649015375,0
"""mint""","""mint + wallet""",2,64021856256,64021856256,0
"""mint""","""ln + mint""",414,39106684786,39106684786,0
"""mint""","""stability_pool""",5,10001018000,0,10001018000
"""mint""","""mint + stability_pool""",1,1458176,2176,1456000


### Check for orphaned `txid`s 
An *orphaned* `txid` is a `txid` that exists in one df but not in the other.  

(Each transaction should have at least one input and one output.)

#### Observation: There are 243 orphaned input `txid`s.

In [9]:
input_orphaned_txids = check_for_orphaned_txids('input', p_in, p_out)

# unique input txid's                       :  137,134
# unique input txid's w/ associated outputs :  136,891
# orphaned input txid's                     :      243

# of orphaned input txid's:


txid,ln,mint,stability_pool,wallet
u32,u32,u32,u32,u32
243,243,0,0,0



# of msats in orphaned input txid's:


txid,ln,mint,stability_pool,wallet
str,i64,i64,i64,i64
null,0,0,0,0


#### Observation: There are 34,975 orphaned output `txid`s.

In [10]:
output_orphaned_txids = check_for_orphaned_txids('output', p_out, p_in)

# unique output txid's                       :  171,848
# unique output txid's w/ associated outputs :  136,891
# orphaned output txid's                     :   34,957

# of orphaned output txid's:


txid,ln,mint,stability_pool,wallet
u32,u32,u32,u32,u32
34957,34956,0,1,0



# of msats in orphaned output txid's:


txid,ln,mint,stability_pool,wallet
str,i64,i64,i64,i64
null,0,0,0,0


Since both the orphaned input `txid`s and the orphaned output `txid`s all have `amount_msat = 0`, we can do a proper inner join without worrying if we're leaving any sats behind.

But it does make one wonder how there are any orphaned `txid`s at all.

#### Observation: Every orphaned `txid` exists in the `txs` table.

In [11]:
iot = (input_orphaned_txids.join(txs, on='txid').group_by('session_index').len().sort(by='session_index'))
oot = (output_orphaned_txids.join(txs, on='txid').group_by('session_index').len().sort(by='session_index'))
print(f"# orphaned input txid's                   : {input_orphaned_txids.count()['txid'][0]:>7,}")
print(f"# orphaned input txid's in the txs table  : {iot.sum()['len'][0]:>7,}")
print(f"# orphaned output txid's                  : {output_orphaned_txids.count()['txid'][0]:>7,}")
print(f"# orphaned output txid's in the txs table : {oot.sum()['len'][0]:>7,}")

# orphaned input txid's                   :     243
# orphaned input txid's in the txs table  :     243
# orphaned output txid's                  :  34,957
# orphaned output txid's in the txs table :  34,957


#### Observation: Over 90% of `session_index`s contain at least 1 orphaned `txid`.

In [12]:
sess_summ = txs.group_by('session_index').agg(pl.col('session_index').count().alias('n_txids'))
print(f"# of session_index's                             : {sess_summ.shape[0]:>7,}")
print(f"# of session_index's containing an orphaned txid : {pl.concat([iot, oot])['session_index'].unique().shape[0]:>7,}")

# of session_index's                             :  36,600
# of session_index's containing an orphaned txid :  33,225


#### Observation: There are plenty of `session_index`s that contain all orphaned `txid`s.

In [13]:
expanded_sess_summ = (
    sess_summ
    .join(iot, on='session_index', how='left').rename({'len': 'n_orphaned_input_txids'})
    .join(oot, on='session_index', how='left').rename({'len': 'n_orphaned_output_txids'})
    .with_columns(pl.sum_horizontal(['n_orphaned_input_txids', 'n_orphaned_output_txids']).alias('n_orphaned_txids'))
    .with_columns((pl.col('n_txids') - pl.col('n_orphaned_txids')).alias('diff'))
)

expanded_sess_summ.filter(pl.col('diff') == 0).sort(by='n_txids', descending=True)

session_index,n_txids,n_orphaned_input_txids,n_orphaned_output_txids,n_orphaned_txids,diff
i64,u32,u32,u32,u32,u32
51328,29,29,null,29,0
46744,11,11,null,11,0
46727,10,10,null,10,0
46740,10,10,null,10,0
…,…,…,…,…,…
45439,1,null,1,1,0
49768,1,null,1,1,0
7568,1,null,1,1,0


#### Observation: Sometimes, a single `session_index` will contain both orphan inputs and orphaned outputs.

In [14]:
(
    expanded_sess_summ
    .filter(
        pl.col('n_orphaned_input_txids') > 0,
        pl.col('n_orphaned_output_txids') > 0,
    )
    .sort(by='diff')
)

session_index,n_txids,n_orphaned_input_txids,n_orphaned_output_txids,n_orphaned_txids,diff
i64,u32,u32,u32,u32,u32
46738,30,27,2,29,1
55920,6,3,1,4,2
58388,6,3,1,4,2
44593,10,1,6,7,3
…,…,…,…,…,…
43757,10,1,2,3,7
46741,17,8,1,9,8
46739,39,3,1,4,35


### Check for rogue `txid`s 
A *rogue* `txid` is one whose sum(input sats) != sum(output sats).

In [15]:
in_cols = ['ln', 'mint', 'stability_pool', 'wallet']
out_cols = ['ln_out', 'mint_out', 'stability_pool_out', 'wallet_out']

rogue_txids = (
    p_in
    .join(p_out, on='txid', suffix='_out')
    .with_columns(
        pl.sum_horizontal(pl.col(in_cols)).alias('in_msats'),
        pl.sum_horizontal(pl.col(out_cols)).alias('out_msats'),
    )
    .filter(
        pl.col('in_msats') != pl.col('out_msats')
    )
    # .sort(by=['in_msats', 'out_msats'])
    .sort(by=['txid'])
    .select(['txid', 'in_msats', 'out_msats'] + in_cols + out_cols)  # reorder columns
)

#### Observation: There are 11 rogue `txid`s. None were expected.

In [16]:
with pl.Config(tbl_rows=20):
    display(rogue_txids)

txid,in_msats,out_msats,ln,mint,stability_pool,wallet,ln_out,mint_out,stability_pool_out,wallet_out
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""054df939f73807fd933c3fa81374c5949ee0bc4a2685151d38342dcd3bcf66d3""",146000,0,null,146000,null,null,null,null,0,null
"""24a69356c8c67a4f35a6d207a500a7be704478692ecdb135e955f7897b829838""",0,545180,null,null,0,null,null,545180,null,null
"""290fc842bd647b353dcd116b96deff02c4d5009c37ff780dbb4eb4a38f2d386b""",726000,0,null,726000,null,null,null,null,0,null
"""2f30bf7351bfc92dcac03f8752556554e8e5c45d1c7ba81eea4a54017251f43d""",5000000000,0,null,5000000000,null,null,null,null,0,null
"""3092897fe709216c046ad33b04bc36c37c402505185967bf11407e011aafbff6""",0,870754,null,null,0,null,null,870754,null,null
"""33e614f0ba832503f8f3f3fac90b1713ca1f6de96cb72e17cd20f0ba890cbed2""",146000,0,null,146000,null,null,null,null,0,null
"""554316f73d8664d336881b8608efe2b1c7c19ca2fcbca606ae1c2d52b1f6a19b""",0,1000000,null,null,0,null,null,1000000,null,null
"""60924d813a8e589b891b68442e0d52f0cf65557905d77da74956de1172b8f711""",0,4998454820,null,null,0,null,null,4998454820,null,null
"""7b238d1263042cfe9eca2cb5a4702fc848e6a2da9acfef0a6eb6a032c3790a70""",5000000000,0,null,5000000000,null,null,null,null,0,null


#### Observation:  All the rogue `txid`s have `kind = 'stability_pool'` on one side of the transaction.
Note the existence of `0` instead of `null` in the `stability_pool` or `stability_pool_out` columns in the table above.

#### Observation: But not all `txid`s with `kind = 'stability_pool'` are rogue.
There is 1 that is orphaned. (`txid = 'ae9525b8a4978073a0aa05854cec3ef339bf61db52ea446e75cd23f756b85169'`)

In [17]:
all_sp_txids = pl.concat([
    p_in.filter(pl.col('stability_pool').is_not_null()), 
    p_out.filter(pl.col('stability_pool').is_not_null())
])

print(f"# stability_pool txid's                   : {all_sp_txids.shape[0]:3}")
print(f"# stability_pool txid's that are rogue    : {all_sp_txids.join(rogue_txids, on='txid').shape[0]:3}")
print(f"# stability_pool txid's that are orphaned : {all_sp_txids.join(p_combined, on='txid', how='anti').shape[0]:3}")

# stability_pool txid's                   :  12
# stability_pool txid's that are rogue    :  11
# stability_pool txid's that are orphaned :   1


### Charts

#### With June 24, 2024 outlier

In [18]:
make_tx_stats_chart('input', p_in, txs, session_times, False)

alt.LayerChart(...)

In [19]:
make_tx_stats_chart('output', p_out, txs, session_times, False)

alt.LayerChart(...)

#### Without June 24, 2024 outlier

In [20]:
make_tx_stats_chart('input', p_in, txs, session_times, True)

alt.LayerChart(...)

In [21]:
make_tx_stats_chart('output', p_out, txs, session_times, True)

alt.LayerChart(...)

### Observation: Date range of txs is from April 6 thru July 30, 2024.

In [22]:
tx_session_times = (
        txs
        .join(session_times, on='session_index')
        .with_columns(
            pl.from_epoch(pl.col('estimated_session_timestamp'), time_unit='s').alias('timestamp')
        )
        .select(['txid', 'session_index', 'timestamp'])
        .sort(by='timestamp')
    )

In [23]:
with pl.Config(tbl_rows=9):
    display(tx_session_times.describe())

statistic,txid,session_index,timestamp
str,str,f64,str
"""count""","""172091""",172091.0,"""172091"""
"""null_count""","""0""",0.0,"""0"""
"""mean""",null,38636.310469,"""2024-06-05 14:34:01.557571"""
"""std""",null,20010.211268,null
"""min""","""00018df854e105d759e6b3df2c139988111da4ade3fc049c2d1f65362a028ad4""",219.0,"""2024-04-06 00:34:18"""
"""25%""",null,21551.0,"""2024-05-09 15:22:14"""
"""50%""",null,39164.0,"""2024-06-06 10:03:54"""
"""75%""",null,55445.0,"""2024-07-02 03:48:34"""
"""max""","""ffff926a59befb44959be264652e91ba1d7f8b50ae2ac90f1ad4d9abfc600cd5""",73627.0,"""2024-07-30 20:39:52"""


### Observation (interesting): the amount of msats minted are always a power of 2

In [24]:
(
    # tx_inputs
    tx_outputs
    .filter(
        pl.col('kind') == 'mint'
    )
    .group_by('amount_msat').len()
    .sort(by='amount_msat')
    .with_columns(
        pl.col('amount_msat').log(base=2).alias('exponent_base_2')
    )
    .rename({'len': 'n_mints'})
    .to_pandas()
)

,amount_msat,n_mints,exponent_base_2
0,1,34843,0.0
1,2,35379,1.0
2,4,35154,2.0
3,8,36219,3.0
4,16,36325,4.0
5,32,36775,5.0
6,64,37299,6.0
7,128,37909,7.0
8,256,78700,8.0
9,512,94973,9.0
